<a href="https://colab.research.google.com/github/tonyjosephsebastians/100DaysMLCode/blob/main/stock_price_calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#!pip install streamlit

     |████████████████████████████████| 9.2 MB 16.4 MB/s 
     |████████████████████████████████| 180 kB 71.2 MB/s 
     |████████████████████████████████| 76 kB 5.5 MB/s 
     |████████████████████████████████| 111 kB 71.9 MB/s 
     |████████████████████████████████| 4.3 MB 55.2 MB/s 
     |████████████████████████████████| 164 kB 82.1 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 126 kB 54.8 MB/s 
     |████████████████████████████████| 792 kB 27.6 MB/s 
     |████████████████████████████████| 374 kB 59.8 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=936a0884ad742e30a7efca307ae5c0e29fb45bf94697373732bc8f40624f64cf
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      

In [2]:
import pandas as pd
import requests
import spacy
import streamlit as st
import os

from spacy import displacy
from bs4 import BeautifulSoup
# import yfinance as yf
# from spacy_streamlit import visualize_ner
import matplotlib


In [3]:
resp = requests.get("https://economictimes.indiatimes.com/markets/stocks/rssfeeds/2146842.cms")
resp

<Response [200]>

In [4]:
soup = BeautifulSoup(resp.content,features='xml' )
soup

<?xml version="1.0" encoding="utf-8"?>
<rss version="2.0" xmlns:atom="http://www.w3.org/2005/Atom"><channel><title>Stocks-Markets-Economic Times</title><link>https://economictimes.indiatimes.com/markets/stocks</link><description>Stock Market News: Latest Stock news and updates on The Economic Times. Find Stock Market Live Updates, BSE, NSE Top Gainers, Losers and more.</description><language>en-gb</language><lastBuildDate>2022-01-06T09:28:26+05:30</lastBuildDate><atom:link href="https://economictimes.indiatimes.com/rssfeeds/2146842.cms" rel="self" type="application/rss+xml"/><copyright>Copyright:(C)2022 Bennett Coleman &amp; Co. Ltd, http://info.indiatimes.com/terms/tou.html</copyright><docs>http://syndication.indiatimes.com/</docs><image><title>Economic Times</title><url>https://img.etimg.com/thumb/msid-76939477,width-600,height-60,quality-100/economictimes.jpg</url><link>https://economictimes.indiatimes.com/markets/stocks</link></image><item><title>Add Wipro, target price Rs 750:  HD

In [5]:
soup.find_all('title')

[<title>Stocks-Markets-Economic Times</title>,
 <title>Economic Times</title>,
 <title>Add Wipro, target price Rs 750:  HDFC Securities </title>,
 <title>Listen In: Get the timing right while buying and selling silver ETFs</title>,
 <title>Buy Eicher Motors, target price Rs 2850:  IIFL </title>,
 <title>Buy Bajaj Finserv, target price Rs 19000:  IIFL </title>,
 <title>Buy Bajaj Finance, target price Rs 8000:  IIFL </title>,
 <title>Buy Voltas, target price Rs 1300:  Yes Securities </title>,
 <title>Buy or Sell: Stock ideas by experts for January 06, 2022</title>,
 <title>Asian shares fall after hawkish Fed minutes</title>,
 <title>Stocks in focus: RBL Bank, NHPC, Motherson Sumi and more</title>,
 <title>SGX Nifty down 120 points; here's what changed for market while you were sleeping</title>,
 <title>Stocks in the news: GAIL, Vedanta, NHPC, RBL Bank and Spandana Sphoorty</title>,
 <title>Nasdaq posts biggest daily drop since Feb after 'hawkish' Fed minutes</title>,
 <title>Hawkish Fed 

In [6]:
nlp = spacy.load("en_core_web_sm")

In [7]:
def extract_text_from_rss(rss_link):
    """
    Parses the XML and extracts the headings from the 
    links in a python list.
    """
    headings = []
    r1 = requests.get('https://economictimes.indiatimes.com/markets/stocks/rssfeeds/2146842.cms')
    r2 = requests.get(rss_link)
    soup1 = BeautifulSoup(r1.content, features='lxml')
    soup2 = BeautifulSoup(r2.content, features='lxml')
    headings1 = soup1.findAll('title')
    headings2 = (soup2.findAll('title'))
    print(headings2)
    headings = headings1 + headings2
    return headings

In [8]:
token_dict = {
    'Org': [],
    'Symbol': [],
    'currentPrice': [],
    'dayHigh': [],
    'dayLow': [],
    'forwardPE': [],
    'dividendYield': []
}

In [16]:
def stock_info(headings):
    """
    Goes over each heading to find out the entities
    and link it with the nifty 500 companies data.
    Extracts the data 
    """
    stocks_df = pd.read_csv("ind_nifty500list.csv")
    for title in headings:
        doc = nlp(title.text)
        for token in doc.ents:
            try:
                if stocks_df['Company Name'].str.contains(token.text).sum():
                    symbol = stocks_df[stocks_df['Company Name'].\
                                        str.contains(token.text)]['Symbol'].values[0]
                    org_name = stocks_df[stocks_df['Company Name'].\
                                        str.contains(token.text)]['Company Name'].values[0]
                    token_dict['Org'].append(org_name)
                    print(symbol+".NS")
                    token_dict['Symbol'].append(symbol)
                    stock_info = yf.Ticker(symbol+".NS").info
                    token_dict['currentPrice'].append(stock_info['currentPrice'])
                    token_dict['dayHigh'].append(stock_info['dayHigh'])
                    token_dict['dayLow'].append(stock_info['dayLow'])
                    token_dict['forwardPE'].append(stock_info['forwardPE'])
                    token_dict['dividendYield'].append(stock_info['dividendYield'])
                else:
                    pass
            except:
                pass
    output_df = pd.DataFrame.from_dict(token_dict,orient='index')
    output_df = output_df.transpose()
    return output_df

In [10]:
user_input = st.text_input("Add your RSS link here!", "https://www.moneycontrol.com/rss/buzzingstocks.xml")

2022-01-06 04:36:14.736 
  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]


In [11]:
fin_headings = extract_text_from_rss(user_input)

[<title>Moneycontrol Buzzing Stocks</title>, <title>Moneycontrol Logo</title>, <title>Buzzing Stocks: NHPC, GAIL India, Mahanagar Gas and other stocks in news today</title>, <title>Over 400 stocks hit 52-week high as continued bull run takes Sensex above 60,000</title>, <title>Gainers  Losers: 10 stocks that moved the most on January 5</title>, <title>Gainers  Losers: 10 stocks that moved the most on January 5</title>, <title>More than 100 stocks post double-digit gains in current rally, outperform Nifty 50âs 7% surge from recent low</title>, <title>HFCL share price rises 13% on partnership with Aprecomm</title>, <title>Brokerages expect HDFC Bank to return up to 32%; resumption of credit card issuance to aid margin recovery</title>, <title>Brokerages expect HDFC Bank to return up to 32%; resumption of credit card issuance to aid margin recovery</title>, <title>AU Small Finance Bank shares jump 6%, brokerages expect 28% upside. Here#39;s why</title>, <title>AU Small Finance Bank shar

In [18]:
output_df = stock_info(fin_headings)

EICHERMOT.NS
BAJAJFINSV.NS
VOLTAS.NS
ASIANPAINT.NS
FEDERALBNK.NS
RBLBANK.NS
NHPC.NS
GAIL.NS
VEDL.NS
NHPC.NS
RBLBANK.NS
SPANDANA.NS
FEDERALBNK.NS
AXISBANK.NS
BSE.NS
HFCL.NS
FEDERALBNK.NS
SRF.NS
INDIGO.NS
PEL.NS
NHPC.NS
MGL.NS
HDFCBANK.NS
HDFCBANK.NS
AUBANK.NS
AUBANK.NS
HATSUN.NS
HATSUN.NS
HATSUN.NS
HATSUN.NS


In [19]:
st.dataframe(output_df)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [20]:
with st.expander("Expand for Financial News!"):
    for h in fin_headings:
        st.markdown("* " + h.text)